# function for data uniform

In [ ]:
seurat_meta_all <- function(obj){
                        if(is.null(obj@misc[['TMI']])){
                             meta_now <- obj@meta.data %>% mutate_if(~!is.numeric(.), ext_list) %>% .[order(.$SampleID),]
                        }else{
                             meta_now <- cbind(obj@meta.data, obj@misc[['TMI']]) %>% mutate_if(~!is.numeric(.), ext_list) %>% .[order(.$SampleID),]
                        }
                        return(meta_now)
}

seurat_object_uniform <- function(obj, barcode_length = 16){
                                proj <- unique(obj$Cohort)
                                if(is.null(obj@misc[['TMI']])){
                                     meta_now <- obj@meta.data %>% mutate_if(~!is.numeric(.), ext_list) %>% .[order(.$SampleID),]
                                }else{
                                     meta_now <- cbind(obj@meta.data, obj@misc[['TMI']]) %>% mutate_if(~!is.numeric(.), ext_list) %>% .[order(.$SampleID),]
                                }
                                umap_emb <- obj@reductions$umap@cell.embeddings %>% .[rownames(meta_now),]
                                counts <- obj@assays$RNA@counts %>% .[,rownames(meta_now)]

                                ## extract SampleID
                                sampleIDs <- obj$SampleID
                                ## generate 16nt length barcode
                                generate_random_barcode <- function(n = 1, length = barcode_length) {
                                  bases <- c("A", "T", "C", "G")
                                  barcodes <- sapply(1:n, function(x) paste0(sample(bases, length, replace = TRUE), collapse = ""))
                                  return(barcodes)
                                }

                                random_barcodes <- generate_random_barcode(n = length(sampleIDs), length = 16)
                                new_barcodes <- paste0(sampleIDs, "_", random_barcodes)

                                # switch the barcode
                                rownames(meta_now) <- new_barcodes
                                rownames(umap_emb) <- new_barcodes
                                colnames(counts) <- new_barcodes

                                # modify meta.data
                                feature_keep <- c('Cohort', 'SampleID', 'SampleType', 'DonorID', 'Sex', 
                                                  'Chemistry', 'Tissue', 
                                                  'gCT', 'mCT', 'oCT')
                                feature_common <- intersect(feature_keep, names(meta_now))
                                meta_now <- meta_now %>% dplyr::select(feature_common)

                                ## Create object for use
                                obj_use <- CreateSeuratObject(counts = counts, project = proj, meta.data = meta_now, min.cells = 0, min.features = 0)
                                obj_use[['umap']] <- CreateDimReducObject(embeddings = umap_emb)
                                return(obj_use)
                            }

# list all file (13 cancer types)

In [ ]:
objList_full <- list.files('/project/sex_cancer/data/step1_preProcess', recursive = T, full.names = T, pattern = 'final')
objList_short <- list.files('/project/sex_cancer/data/step1_preProcess', recursive = T, full.names = F, pattern = 'final')
objList_short

In [ ]:
feature_keep <- c('Cohort', 'SampleID', 'SampleType', 'DonorID', 'Sex', 'Chemistry', 'Tissue', 'gCT', 'mCT', 'oCT')

# obj1: AML_Galen2019

In [ ]:
obj <- readRDS(objList_full[1])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))

obj_use <- seurat_object_uniform(obj, barcode_length = 12)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.AML_Galen2019.rds')

# obj2: BLCA_Gouin2021

In [ ]:
obj <- readRDS(objList_full[2])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))

obj@meta.data <- obj@meta.data %>% transform(Cohort = 'BLCA_Gouin2021', Tissue = 'Bladder', Chemistry = "10x 3' v3")
obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.BLCA_Gouin2021.rds')

# obj3: ccRCC_Hu2024

In [ ]:
obj <- readRDS(objList_full[3])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
obj@meta.data <- obj@meta.data %>% transform(Tissue = 'Kidney', Chemistry = "10x GEX")
meta_now <- obj@meta.data
obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.ccRCC_Hu2024.rds')

# obj4: CRC_Pelka2021

In [ ]:
obj <- readRDS(objList_full[4])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))
obj@meta.data <- obj@meta.data %>% transform(Tissue = 'Colon')

obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.CRC_Pelka2021.rds')

# obj5: ESCA_Zhang2021

In [ ]:
obj <- readRDS(objList_full[5])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))
obj@meta.data <- obj@meta.data %>% transform(Tissue = 'Esophagus', Chemistry = "10x 5'")

obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.ESCA_Zhang2021.rds')

# obj6: FL_Han2022

In [ ]:
obj <- readRDS(objList_full[6])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))

obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.FL_Han2022.rds')

# obj7: GBM_Spitzer2025

In [ ]:
obj <- readRDS(objList_full[7])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))
obj@meta.data <- obj@meta.data %>% transform(Tissue = "Brain", Chemistry = "10x 3' v3")

obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.GBM_Spitzer2025.rds')

# obj8: GC_Kumar2022

In [ ]:
obj <- readRDS(objList_full[8])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))

obj@meta.data <- obj@meta.data %>% transform(Tissue = 'Stomach')
obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.GC_Kumar2022.rds')

# obj9: HNSCC_Choi2023

In [ ]:
obj <- readRDS(objList_full[9])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))
obj@meta.data <- obj@meta.data %>% transform(Tissue = "Oral cavity", Chemistry = "10x 3' v2")

obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.HNSCC_Choi2023.rds')

# obj10: LIHC_Xue2022

In [ ]:
obj <- readRDS(objList_full[10])
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))

obj@meta.data <- obj@meta.data %>% transform(DonorID = SampleID, Tissue = "Liver", SampleType = 'tumor')

obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.LIHC_Xue2022.rds')

# obj11: NSCLC_Salcher2022

In [ ]:
obj <- readRDS(objList_full[11])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))

obj@meta.data <- obj@meta.data %>% transform(Tissue = 'Lung')

obj_use <- seurat_object_uniform(obj, barcode_length = 17)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.NSCLC_Salcher2022.rds')

# obj12: PDAC_Hwang2022

In [ ]:
obj <- readRDS(objList_full[12])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))

obj@meta.data <- obj@meta.data %>% transform(DonorID = SampleID, Tissue = "Pancreas")

obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.PDAC_Hwang2022.rds')

# obj13: THCA_Pu2021

In [ ]:
obj <- readRDS(objList_full[13])
obj <- UpdateSeuratObject(obj)
obj

## uniform

In [ ]:
meta_now <- seurat_meta_all(obj)
# check meta.data
list('keep' = intersect(feature_keep, names(meta_now)),
     'miss' = setdiff(feature_keep, names(meta_now)),
     'TMI' = setdiff(names(meta_now), feature_keep))

obj@meta.data <- obj@meta.data %>% transform(Tissue = "Thyroid", Chemistry = "10x v3")

obj_use <- seurat_object_uniform(obj, barcode_length = 16)
obj_use
obj_use@meta.data %>% head(n = 2)

## UMAP

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
select <- 'umap'
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "gCT", label = T, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "mCT", label = F, label.size = 6, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "oCT", label = F, colors_use = pal_igv("default")(51))|
DimPlot_scCustom(obj_use, reduction = select, pt.size = 1, group.by = "SampleType", label = F, label.size = 8, colors_use = pal_igv("default")(51))

## Statistics

In [ ]:
obj_use@meta.data[,c('SampleID', 'SampleType', 'Sex')] %>% .[!duplicated(.$SampleID),] %$% table(.$SampleType, .$Sex)
obj_use@meta.data %$% table(.$SampleType)

## save for use

In [ ]:
saveRDS(obj_use, 'obj.THCA_Pu2021.rds')